In [1]:
import os
import sys
sys.path.append('../')
import utils.csv as csv
import utils.plot as plot

In [7]:
PATH='D:\\Deutschland\\FUB\\master_thesis\\data'
DATA_DIR = os.path.join(PATH, 'gee', 'output', 'validation_grid_daily_padding')
LABEL_CSV = 'label_val_grid.csv'
METHOD = 'classification'
MODEL = 'transformer'
UID = '8pure9'
MODEL_NAME = MODEL + '_' + UID
LABEL_PATH = os.path.join(PATH, 'ref', 'validation', LABEL_CSV)
MODEL_PATH = f'../outputs/models/{METHOD}/01/{MODEL_NAME}.pth'

In [3]:
 # *************************change class here*************************
classes = ['Spruce', 'Sliver Fir','Douglas Fir','Pine','Oak','Red Oak','Beech','Sycamore','Others']
# *******************************************************************
ref = csv.load(f'../outputs/csv/{METHOD}/01/{MODEL_NAME}_ref.csv', 'id')
pred = csv.load(f'../outputs/csv/{METHOD}/01/{MODEL_NAME}_pred.csv', 'id')

In [8]:
plot.draw_confusion_matrix(ref, pred, classes, MODEL_NAME)

<Figure size 1000x1000 with 0 Axes>